In [2]:
import sys
sys.path.append("..")
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from rasterstats import zonal_stats
import rasterio
import contextily as cx
from rasterio.plot import show
#import osmnx as ox
from affine import Affine
import rioxarray as rx
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from pathlib import Path
import fiona



#see all columns in tables
pd.set_option('display.max_columns', None)


%load_ext autoreload
%autoreload 2

In [5]:
from src.data.make_dataset import *
from src.features.build_features import *

Reading in municipal and regional boundaries...
Reading in land parcel database data...
Reading in public excluded land data...
Reading in additional data layers...
Wetlands...
Watersheds...
Wellhead protection areas...
Activity use limitation areas...
ParkServe data...


### Identify muni/munis of interest, create a shapefile for the muni

Then create land use and land cover layers of interest based on the municipal boundary mask (speeds up processes)

In [6]:
#select a town name, create a gdf of that town boundary
town_name = 'Franklin'

muni_shp = munis.loc[munis['municipal'] == town_name]

### Pull base geographies for analysis
- Get municipal ROW and fishnet (tesselation) layers from geodatabases 
- Clean up those gdfs to include only the relevant info

In [10]:
#read in relevant data layers - at some point this will have to be automated but for now we can use this

fishnets_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Fishnets.gdb'

for layer_name in fiona.listlayers(fishnets_gdb):
    if town_name in layer_name:
        town_tess = layer_name

muni_tess = gpd.read_file(fishnets_gdb, layer=town_tess)
#muni_tess = gpd.read_file(muni_tess_gdb, layer='muni_tessellation')

row_gdb = 'K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\ROW_model_output.gdb'

for layer_name in fiona.listlayers(row_gdb):
    if town_name in layer_name:
        town_row_layer = layer_name

town_row = gpd.read_file(row_gdb, layer=town_row_layer)


#### Parcels

In [86]:
#prepare parcel layer wtih parcels and row segments
town_parcels = get_landuse_data(town_name, mapc_lpd)
town_parcels['muni'] = town_name
town_parcels['type'] = 'parcel'

#set up row layer

#don't need most field names
town_row = town_row[['poly_typ', 'geometry']]

#create ID field
town_row.insert(0, 'parloc_id', range(10000, 10000 + len(town_row)))


#create type and muni fields that will be consistent with the parcels fields
town_row['type'] = 'ROW segment'
town_row['muni'] = town_name

#merge together ROW parcels with parcel data from 3a - final dataset for spatial operations
town_parcels_row = pd.concat([town_parcels, town_row])

#### Fishnets

In [12]:
#clip tesselation to the franklin boundary
muni_tess = muni_tess.clip(muni_shp)
muni_tess['sqm'] = muni_tess['geometry'].area

`town_parcels_row` and `town_tess` are the two base geographies for this analysis.

## Inidicator Layer Prep 
To improve run time, for some indicators it is better to read in here with a municipal mask 

In [56]:
#create an imperviousness layer
#first read in lclu with a muni shapefile mask

from shapely.validation import make_valid
ms4_model_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Model.gdb'

lclu = gpd.read_file(ms4_model_gdb, layer="lclu_simplify_all_mapc", mask=muni_shp)

#fix any geometry issues
print('Validating land use geometry...')
lclu.geometry = lclu.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)
lclu = lclu.clip(muni_shp)

#select only for imperviousness land cover codes
lclu_impervious = lclu.loc[lclu['covercode'] == 2]

#fix any geometry issues
print('Validating imperviousness geometry...')
lclu_impervious.geometry = lclu_impervious.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

#tree canopy using land use
tree_canopy_covernames = ['Deciduous Forest', 'Evergreen Forest', 'Palustrine Forested Wetland']
lclu_treecanopy = lclu.loc[lclu['covername'].isin(tree_canopy_covernames)]
lclu_treecanopy.geometry = lclu_treecanopy.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

Validating land use geometry...
Validating imperviousness geometry...


In [ ]:
#make a layer that gives us impervious surfaces but distinguishes between impervious land cover vs rooftops

#read in structures for the muni
building_structures_gdb = 'K:\\DataServices\\Datasets\\MassGIS\\Facilities_Structures\\Building_Structures\\Output\\structures.gdb'
building_structures = gpd.read_file(building_structures_gdb, layer='STRUCTURES_POLY', mask=muni_shp)

#add a type field
building_structures['type'] = 'rooftop'

#erase structures from the imperviousness lclu layer
imperv_cover = lclu_impervious.overlay(building_structures, how='difference')

#now we just have land cover (not rooftops) that are imperviousness
#add a type field
imperv_cover['type'] = 'land cover'

#join back together with the 'type' field being the distinguisher btwn land cover and rooftops
imperviousness_by_type = pd.concat([building_structures[['type', 'geometry']],
                                    imperv_cover[['type', 'geometry']]
                                    ])

#dissolve by type
imperviousness_by_type = imperviousness_by_type.dissolve(by='type')

#add a "layer" field for the model
imperviousness_by_type['layer'] = 'impervious'

In [ ]:
#get tree "need" layer
muni_tree_need = tree_score(muni_shp, lclu_treecanopy)
muni_tree_need.head()

,bg20_id,sqm_tree,pct_tree,geometry,rnk_tree,tree_need
3,250214422042,5.246121e+05,0.450436,"POLYGON ((207975.105 869760.394, 208132.262 86...",0.263158,Moderately low tree canopy
4,250214422041,2.710530e+05,0.367818,"POLYGON ((208147.115 869921.200, 208187.953 87...",0.105263,Very low tree canopy
6,250214421014,3.095390e+05,0.421313,"POLYGON ((207503.177 870795.283, 207894.273 87...",0.157895,Very low tree canopy
7,250214421013,1.751049e+05,0.288130,"POLYGON ((208074.155 871421.245, 208246.479 87...",0.052632,Very low tree canopy
9,250214421041,1.590468e+06,0.421351,"POLYGON ((206076.131 872574.385, 206321.060 87...",0.210526,Moderately low tree canopy


In [7]:
#ms4_model_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Model.gdb'
#imperv = gpd.read_file(ms4_model_gdb, layer='imperv_simplify_all_mapc', mask=muni_shp)

## Parcel Analysis

In [22]:
print('public land...')
#does it overlap with public land?
public = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='parloc_id',
                                        parcel_data=town_parcels_row, 
                                        join_data=public_land, 
                                        field='USE_DESC',
                                        layer_name='pblc',
                                        overlap=0.05)

print('imperviousness...')
#how much imperviousness is on site?
imperv = calculate_imperviousness(town_parcels_row, 
                                  'parloc_id', 
                                  imperv_cover, 
                                  building_structures)


print('park serve need...')
#does it overlap with Trust for Public Land's "Park Serve" priority areas from 2022
parkserve = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='parloc_id',
                                        parcel_data=town_parcels_row, 
                                        join_data=parkserve_data, 
                                        field='ParkRank',
                                        layer_name='parks',
                                        overlap=0.05)

print('wetlands...')
#does it overlap with wetlands?
wetlands_ovlp = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='parloc_id',
                                            parcel_data=town_parcels_row, 
                                            join_data=wetlands, 
                                            field='IT_VALDESC',
                                            layer_name='wtlnds',
                                            overlap=0.05)

print('wpa...')
#does it overlap with wellhead protection areas?
wpa_z1_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='parloc_id',
                                            parcel_data=town_parcels_row, 
                                            join_data=zone1_wpa, 
                                            field='SUPPLIER',
                                            layer_name='z1_wpa',
                                            overlap=0.05)

wpa_z2_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='parloc_id',
                                            parcel_data=town_parcels_row, 
                                            join_data=zone2_wpa, 
                                            field='SUPPLIER',
                                            layer_name='z2_wpa',
                                            overlap=0.05)


wpa_interim_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='parloc_id',
                                            parcel_data=town_parcels_row, 
                                            join_data=interim_wpa, 
                                            field='SUPPLIER',
                                            layer_name='int_wpa',
                                            overlap=0.05)
print('aul...')
#does it have an activity use limitation area within it?
aul_ovlp = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='parloc_id',
                                            parcel_data=town_parcels_row, 
                                            points=True,
                                            join_data=aul, 
                                            field='NAME',
                                            layer_name='aul',
                                            overlap=0.05)

print('watershed...')
#what watershed does it overlap with?
watershed = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='parloc_id',
                                            parcel_data=town_parcels_row, 
                                            join_data=major_basins, 
                                            field='NAME',
                                            layer_name='wtshd',
                                            overlap=0.05)

print('tree need...')
#is it in a block group with tree need?
tree_need = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='parloc_id',
                                        parcel_data=town_parcels_row, 
                                        join_data=muni_tree_need, 
                                        field='rnk_tree',
                                        layer_name='tree',
                                        overlap=0.05)

public land...
imperviousness...
park serve need...
wetlands...
wpa...
aul...
watershed...
tree need...


## Tesselation Analysis

In [ ]:
#then join parcel data to the tesselations. Each tessel may have more than one associated 
#this might be more relevant later
muni_tess_parcels = muni_tess.overlay(town_parcels_row, how='intersection') 

In [24]:
#does it overlap with public land
print("Calculating public land overlap...")
public_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=public_land, 
                                            field='GROUP_',
                                            layer_name='pblc',
                                            overlap=0.05)

print("Calculating right of way overlap...")
row_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=town_row, 
                                            field='ROW_ID',
                                            layer_name='row',
                                            overlap=0.05)

print("Calculating imperviousness overlap...")
imperv = calculate_imperviousness(muni_tess, 
                                  'GRID_ID', 
                                  imperv_cover, 
                                  building_structures)


print("Calculating park serve overlap...")
#does it overlap with Trust for Public Land's "Park Serve" priority areas from 2022
parkserve_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=parkserve_data, 
                                        field='ParkRank',
                                        layer_name='parks',
                                        overlap=0.05)

print("Calculating wetland overlap...")
#does it overlap with wetlands?
wetlands_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=wetlands, 
                                            field='IT_VALDESC',
                                            layer_name='wtlnds',
                                            overlap=0.05)
print("Calculating wpa overlap...")
#does it overlap with wellhead protection areas?
wpa_z1_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=zone1_wpa, 
                                            field='SUPPLIER',
                                            layer_name='z1_wpa',
                                            overlap=0.05)

wpa_z2_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=zone2_wpa, 
                                            field='SUPPLIER',
                                            layer_name='z2_wpa',
                                            overlap=0.05)


wpa_interim_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=interim_wpa, 
                                            field='SUPPLIER',
                                            layer_name='int_wpa',
                                            overlap=0.05)

print("Calculating aul overlap...")
#does it have an activity use limitation area within it?
aul_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            points=True,
                                            join_data=aul, 
                                            field='NAME',
                                            layer_name='aul',
                                            overlap=0.05)

print("Calculating watershed overlap...")
#what watershed does it overlap with?
watershed_tess = calculate_suitability_criteria(how='overlap_sjoin', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=major_basins, 
                                            stats='first',
                                            field='NAME',
                                            layer_name='wtshd')

print("Calculating tree need overlap...")
#is it in a block group with tree need?
tree_need = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=muni_tree_need, 
                                        field='rnk_tree',
                                        layer_name='tree',
                                        overlap=0.05)


Calculating public land overlap...
Calculating right of way overlap...
Calculating imperviousness overlap...
Calculating park serve overlap...
Calculating wetland overlap...
Calculating wpa overlap...
Calculating aul overlap...
Calculating watershed overlap...
Calculating tree need overlap...
